In [50]:
#Importing the liberaries
import pandas as pd
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [51]:
feedback_df = pd.read_excel("feedback.xlsx")

In [52]:
# Exploratory Data Analysis

In [53]:
# Remove white spaces from Date fields
feedback_df.replace('\n','', regex=True, inplace = True)

In [54]:
feedback_df.head()

,S.No.,Date,Comment,Tags
0,1,‎01‎ ‎Apr‎ ‎2017,What a complete and utter shambles. My wife ra...,#Loan#Refund#Overpayment
1,2,‎01‎ ‎Apr‎ ‎2017,Another absolutely feckless conversation with ...,#Loan#Payments#SLC
2,3,‎15‎ ‎May‎ ‎2017,Paid off student loan 6 years ago - then sudde...,#Loan#StudentLoan
3,4,‎24‎ ‎Sep‎ ‎2018,"If I could ""leave no star"" - as a previous rev...",#SLC#Loan#customer
4,5,‎17‎ ‎Sep‎ ‎2018,If there were no star they would deserve it. I...,#MaintenanceLoan#Loan#HMRC


In [55]:
feedback_df1 = feedback_df.drop('S.No.', axis=1)

In [56]:
# No. of rows and columns
feedback_df1.shape
# Size of the dataset is small for building the model

(96, 3)

In [57]:
feedback_df1.info()
# Is the date having Null values accepted? What does Null value in Date field mean? 
# What should this Null date replaced with?
# or should the Null date be removed?  - 26 rows with Null date

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
Date       43 non-null object
Comment    96 non-null object
Tags       95 non-null object
dtypes: object(3)
memory usage: 2.3+ KB


In [58]:
# Remove missing values
feedback_df2 = feedback_df1.dropna().reset_index(drop=True)
feedback_df2.isnull().sum()

Date       0
Comment    0
Tags       0
dtype: int64

In [59]:
comment_df = feedback_df2['Comment']
comment_list = comment_df.tolist()

In [60]:
# import nltk
# nltk.download_gui()
# (or)
# Download from anaconda prompt - python -m nltk.downloader brown and python -m nltk.downloader abc

In [61]:
from nltk import sent_tokenize, word_tokenize, ne_chunk
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [62]:
# text = "What a complete and utter shambles. My wife rang this comical lot this morning to obtain a balance on what we think is now an over paid loan. Having gone through the extensive DPA/Security check, she was then told that they can't give balances over the phone! So what is this line for and why wasn't she made aware at the beginning of the call. We now have to create an online account (which takes 24 hours) and then call back if a refund is due. So let me get this right, you can't discuss balances over the phone yet you an discuss overpayments!"

In [74]:
pre_processed_text = []
for text in comment_list:
    sentences = []
    pre_processed_sent = []
    # Sentence segmentation :
    sentences = sent_tokenize(text)
    for sentence in sentences:
        # Tokenizing
        words = word_tokenize(sentence)
        # Removong stop words
        stop_words = set(stopwords.words('english'))
        stop_words = stop_words.union(string.punctuation)
        clean_words = [word for word in words if not word in stop_words]
        # Identify Parts of speech - noun, verb, adjective
        tagged_words = pos_tag(clean_words)
        # Stemming and Lemmatization - Identify root of the word and dictionary usage
        stemmer = PorterStemmer()
        stemmed_words = [stemmer.stem(clean_word) for clean_word in clean_words]
        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [ lemmatizer.lemmatize(w, pos = 'v') for w in stemmed_words ]
#         print (lemmatized_words)
        # Chunking - finding relation
#         ner = ne_chunk(lemmatized_words)
        pre_processed_sent.append(lemmatized_words)
    pre_processed_text.append(pre_processed_sent)
        # Named Entity Recognition
        # TF - IDF : Term Frequency - Inverse Document Frequency - calculate weight:

In [85]:
# Find parity using Lexican based approach
from textblob import TextBlob
pre_processed_text = []
for text in comment_list:
    blob = TextBlob(text)
    pre_processed_sent = []
    for sentence in blob.sentences:
#         print(sentence.sentiment.polarity)
        pre_processed_sent.append(sentence.sentiment.polarity)
    # Need to sum the polarity
    pre_processed_text.append(pre_processed_sent)
# print(pre_processed_text)

In [87]:
feedback_df2['comment_processed'] = pre_processed_text
# feedback_df2.drop('Comment', axis=1, inplace=True)
feedback_df2.head()

,Date,Tags,comment_processed
0,‎01‎ ‎Apr‎ ‎2017,#Loan#Refund#Overpayment,"[0.05, 0.5, 0.0, 0.25, -0.0625, 0.357142857142..."
1,‎01‎ ‎Apr‎ ‎2017,#Loan#Payments#SLC,"[0.2, -0.3125, 0.175, 0.0, 0.0]"
2,‎15‎ ‎May‎ ‎2017,#Loan#StudentLoan,"[0.0, -0.1, 0.4375, 0.0625, 0.2]"
3,‎24‎ ‎Sep‎ ‎2018,#SLC#Loan#customer,"[-0.16666666666666666, -0.16666666666666666, 0..."
4,‎17‎ ‎Sep‎ ‎2018,#MaintenanceLoan#Loan#HMRC,"[0.0, 0.0, 0.0, 0.0, -0.5, 0.2, 0.0, 0.0, 0.0,..."
